In [1]:
import numpy as np
import pandas as pd
import os

# for matlab files
import scipy.io as sio

In [2]:
# Dataset downloaded from https://csegroups.case.edu/bearingdatacenter/pages/download-data-file
# Normal baseline data

# !wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/97.mat
# !wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/98.mat
# !wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/99.mat
# !wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/100.mat

In [7]:
# Normal baseline data or healthy data
# extracting the matlab files from the html in the webpage
# awk -F(field separator)

!for url in `curl -s https://csegroups.case.edu/bearingdatacenter/pages/normal-baseline-data | grep mat | grep http | \
    awk -F 'href="' '{print $2}' | awk -F '">' '{print $1}'`;do wget $url; done;

--2020-03-08 10:21:34--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/97.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/97.mat [following]
--2020-03-08 10:21:34--  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/97.mat
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3903344 (3.7M)
Saving to: ‘97.mat’

100%[======================================>] 3,903,344    836KB/s   in 4.6s   

2020-03-08 10:21:41 (836 KB/s) - ‘97.mat’ saved [3903344/3903344]

--2020-03-08 10:21:41--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/98.mat
Resolving csegroups.case.ed

In [8]:
!ls -lart

total 34120
-rw-r----- 1 dsxuser dsxuser  7770624 Sep  7  2011 100.mat
-rw-r----- 1 dsxuser dsxuser  3903344 Sep  7  2011 97.mat
-rw-r----- 1 dsxuser dsxuser  7742720 Sep  7  2011 98.mat
-rw-r----- 1 dsxuser dsxuser 15503928 Sep  7  2011 99.mat
drwx------ 1 dsxuser dsxuser     4096 Mar  8 10:07 ..
drwxr-x--- 2 dsxuser dsxuser     4096 Mar  8 10:22 .


In [9]:
# storing the healthy data in dir

!mkdir cwr_healthy
!mv *.mat cwr_healthy

In [10]:
!ls -lart cwr_healthy

total 34120
-rw-r----- 1 dsxuser dsxuser  7770624 Sep  7  2011 100.mat
-rw-r----- 1 dsxuser dsxuser  3903344 Sep  7  2011 97.mat
-rw-r----- 1 dsxuser dsxuser  7742720 Sep  7  2011 98.mat
-rw-r----- 1 dsxuser dsxuser 15503928 Sep  7  2011 99.mat
drwxr-x--- 3 dsxuser dsxuser     4096 Mar  8 10:23 ..
drwxr-x--- 2 dsxuser dsxuser     4096 Mar  8 10:23 .


In [15]:
sio.loadmat('cwr_healthy/100.mat')

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Fri Jan 28 11:25:48 2000',
 '__version__': '1.0',
 '__globals__': [],
 'X100_DE_time': array([[ 0.01460308],
        [ 0.05444862],
        [ 0.10764554],
        ...,
        [-0.02357354],
        [ 0.00521538],
        [ 0.04777292]]),
 'X100_FE_time': array([[ 0.19292182],
        [ 0.16436364],
        [ 0.09081091],
        ...,
        [ 0.10930182],
        [ 0.05218545],
        [-0.00452   ]]),
 'X100RPM': array([[1725]], dtype=uint16)}

In [2]:
# Since all the files are not of the same shape
def read_folder(folder):
    data = 'dummy'
    skip = False
    for file in os.listdir(folder):
        file_id = file[:-4]
        mat_file_dict = sio.loadmat(folder+file)
        del data
        for key, value in mat_file_dict.items():
            if 'DE_time' in key or 'FE_time' in key:
                a = np.array(mat_file_dict[key])
                try:
                    data
                except NameError:
                    data = a
                else:
                    if (data.shape[0] != a.shape[0]):
                        print('skipping ' + file_id)
                        skip = True
                        continue
                    data = np.hstack((data,a))
        if skip:
            skip=False
            continue
        id = np.repeat(file_id,data.shape[0])
        id.shape = (id.shape[0],1)
        data = np.hstack((id,data))
        if data.shape[1] == 2:
            zeros = np.repeat(float(0),data.shape[0])
            zeros.shape =(data.shape[0],1)
            data = np.hstack((data,zeros))
        try:
            result
        except NameError:
            result = data
        else:
            result = np.vstack((result,data))
    return result

In [20]:
read_folder('./cwr_healthy/')

skipping 99
skipping 99


array([['100', '0.014603076923076923', '0.19292181818181817'],
       ['100', '0.05444861538461539', '0.16436363636363635'],
       ['100', '0.10764553846153846', '0.09081090909090908'],
       ...,
       ['98', '-0.09909230769230769', '-0.007601818181818181'],
       ['98', '-0.10827138461538462', '0.0402690909090909'],
       ['98', '-0.07092923076923077', '0.06101999999999999']],
      dtype='<U32')

In [21]:
result_healthy = read_folder('./cwr_healthy/')

skipping 99
skipping 99


In [22]:
# create a pandas dataframe from the above
pdf = pd.DataFrame(result_healthy)

In [25]:
pdf.head(10)

,0,1,2
0,100,0.014603076923076923,0.19292181818181817
1,100,0.05444861538461539,0.16436363636363635
2,100,0.10764553846153846,0.09081090909090908
3,100,0.13372246153846154,0.08649636363636364
4,100,0.11265230769230769,0.09923454545454545
5,100,0.08240307692307691,0.09307090909090908
6,100,0.08699261538461538,0.08197636363636364
7,100,0.11056615384615384,0.03307818181818182
8,100,0.12767261538461538,0.01171090909090909
9,100,0.11348676923076922,0.02218909090909091


In [26]:
# writing to csv file
pdf.to_csv('result_healthy_pandas.csv', header=False, index=True)

In [28]:
!head -n 5 'result_healthy_pandas.csv'

0,100,0.014603076923076923,0.19292181818181817
1,100,0.05444861538461539,0.16436363636363635
2,100,0.10764553846153846,0.09081090909090908
3,100,0.13372246153846154,0.08649636363636364
4,100,0.11265230769230769,0.09923454545454545


In [7]:
# Now lets get the faulty data
# instead of using !wget which will be tedious, we will get the data from html

# 12k-drive-end-bearing-fault-data
!for url in `curl -s https://csegroups.case.edu/bearingdatacenter/pages/12k-drive-end-bearing-fault-data | grep mat | grep http \
    | awk -F 'href="' '{print $2}' | awk -F '">' '{print $1}'`; do wget $url; done;
    
# 48k Drive End Bearing Fault Data
!for url in `curl -s https://csegroups.case.edu/bearingdatacenter/pages/48k-drive-end-bearing-fault-data | grep mat | grep http \
    | awk -F 'href="' '{print $2}' | awk -F '">' '{print $1}'`; do wget $url; done;
    
# 12k Fan End Bearing Fault Data
!for url in `curl -s https://csegroups.case.edu/bearingdatacenter/pages/12k-fan-end-bearing-fault-data | grep mat | grep http \
    | awk -F 'href="' '{print $2}' | awk -F '">' '{print $1}'`; do wget $url; done;

--2020-03-08 12:56:21--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/105.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/105.mat [following]
--2020-03-08 12:56:21--  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/105.mat
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2910768 (2.8M)
Saving to: ‘105.mat’

100%[======================================>] 2,910,768   9.01MB/s   in 0.3s   

2020-03-08 12:56:21 (9.01 MB/s) - ‘105.mat’ saved [2910768/2910768]

--2020-03-08 12:56:21--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/118.mat
Resolving csegroups.

Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2928192 (2.8M)
Saving to: ‘158.mat’

100%[======================================>] 2,928,192   8.83MB/s   in 0.3s   

2020-03-08 12:56:26 (8.83 MB/s) - ‘158.mat’ saved [2928192/2928192]

--2020-03-08 12:56:26--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/107.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/107.mat [following]
--2020-03-08 12:56:26--  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/107.mat
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
HTTP request sent, awaiting response... 200 OK


Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/160.mat [following]
--2020-03-08 12:56:37--  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/160.mat
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2931672 (2.8M)
Saving to: ‘160.mat’

100%[======================================>] 2,931,672   4.04MB/s   in 0.7s   

2020-03-08 12:56:37 (4.04 MB/s) - ‘160.mat’ saved [2931672/2931672]

--2020-03-08 12:56:37--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/169.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|

Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2924712 (2.8M)
Saving to: ‘199.mat’

100%[======================================>] 2,924,712   4.47MB/s   in 0.6s   

2020-03-08 12:56:48 (4.47 MB/s) - ‘199.mat’ saved [2924712/2924712]

--2020-03-08 12:56:48--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/172.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/172.mat [following]
--2020-03-08 12:56:48--  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/172.mat
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
HTTP request sent, awaiting response... 200 OK


Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/223.mat [following]
--2020-03-08 12:56:57--  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/223.mat
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2921232 (2.8M)
Saving to: ‘223.mat’

100%[======================================>] 2,921,232   2.35MB/s   in 1.2s   

2020-03-08 12:56:59 (2.35 MB/s) - ‘223.mat’ saved [2921232/2921232]

--2020-03-08 12:56:59--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/235.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 302 Foun

...

Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/294.mat [following]
--2020-03-08 12:58:22--  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/294.mat
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2908440 (2.8M)
Saving to: ‘294.mat’

100%[======================================>] 2,908,440   3.62MB/s   in 0.8s   

2020-03-08 12:58:23 (3.62 MB/s) - ‘294.mat’ saved [2908440/2908440]

--2020-03-08 12:58:23--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/298.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 302 Foun

Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2921664 (2.8M)
Saving to: ‘284.mat’

100%[======================================>] 2,921,664   5.61MB/s   in 0.5s   

2020-03-08 12:58:32 (5.61 MB/s) - ‘284.mat’ saved [2921664/2921664]

--2020-03-08 12:58:32--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/296.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/296.mat [following]
--2020-03-08 12:58:32--  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/296.mat
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
HTTP request sent, awaiting response... 200 OK


HTTP request sent, awaiting response... 302 Found
Location: https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/286.mat [following]
--2020-03-08 12:58:43--  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/286.mat
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2930472 (2.8M)
Saving to: ‘286.mat’

100%[======================================>] 2,930,472   3.50MB/s   in 0.8s   

2020-03-08 12:58:44 (3.50 MB/s) - ‘286.mat’ saved [2930472/2930472]

--2020-03-08 12:58:44--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/313.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Da

HTTP request sent, awaiting response... 200 OK
Length: 2928496 (2.8M)
Saving to: ‘277.mat’

100%[======================================>] 2,928,496   4.03MB/s   in 0.7s   

2020-03-08 12:58:53 (4.03 MB/s) - ‘277.mat’ saved [2928496/2928496]

--2020-03-08 12:58:53--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/289.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/289.mat [following]
--2020-03-08 12:58:53--  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/289.mat
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2908440 (2.8M)
Saving to: ‘289.mat’

100%[====================================

Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/292.mat [following]
--2020-03-08 12:59:00--  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/292.mat
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2917248 (2.8M)
Saving to: ‘292.mat’

100%[======================================>] 2,917,248   6.26MB/s   in 0.4s   

2020-03-08 12:59:01 (6.26 MB/s) - ‘292.mat’ saved [2917248/2917248]

--2020-03-08 12:59:01--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/317.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 302 Foun

In [8]:
!mkdir cwr_faulty

mkdir: cannot create directory ‘cwr_faulty’: File exists


In [9]:
!ls -lart | head -n 10

total 638412
-rw-r----- 1 dsxuser dsxuser  2910768 Sep  7  2011 105.mat
-rw-r----- 1 dsxuser dsxuser  2928192 Sep  7  2011 106.mat
-rw-r----- 1 dsxuser dsxuser  2931672 Sep  7  2011 107.mat
-rw-r----- 1 dsxuser dsxuser  2950416 Sep  7  2011 108.mat
-rw-r----- 1 dsxuser dsxuser  3903344 Sep  7  2011 109.mat
-rw-r----- 1 dsxuser dsxuser  7770624 Sep  7  2011 111.mat
-rw-r----- 1 dsxuser dsxuser  7779920 Sep  7  2011 110.mat
-rw-r----- 1 dsxuser dsxuser  2942112 Sep  7  2011 118.mat
-rw-r----- 1 dsxuser dsxuser  7770624 Sep  7  2011 112.mat
ls: write error: Broken pipe


In [10]:
!mv *.mat cwr_faulty/

In [ ]:
result_faulty = read_folder('./cwr_faulty/')

skipping 217
skipping 217
